In [ ]:
%load_ext autoreload
%autoreload 2
%aimport utils_1_1

import pandas as pd
import numpy as np
import altair as alt
from altair_saver import save
import datetime
import dateutil.parser
from os.path import join

from constants_1_1 import SITE_FILE_TYPES
from utils_1_1 import (
    get_site_file_paths,
    get_site_file_info,
    get_site_ids,
    get_visualization_subtitle,
    get_country_color_map,
    apply_theme,
)
from web import for_website

alt.data_transformers.disable_max_rows(); # Allow using rows more than 5000

In [ ]:
df = pd.read_csv(join("..", "data", "Phase2.1", "temporal", "charlson", "charlson_countrylevel.csv"))
df.head()

df = df.loc[~df["country"].isin(["SINGAPORE"])]
df["country"] = df["country"].apply(lambda name: name if name in ["USA", "UK"] else name[0] + name.lower()[1:])

cs_df = pd.DataFrame()
for (country, setting), country_setting_df in df.groupby(by=["country", "setting"]):    
    cs_df = cs_df.append({
        "country": country,
        "setting": setting,
        "charlson_early": country_setting_df.loc[country_setting_df["wave"] == "early"].iloc[0]["charlson"],
        "charlson_late": country_setting_df.loc[country_setting_df["wave"] == "late"].iloc[0]["charlson"]
    }, ignore_index=True)
df = cs_df
df["difference"] = df["charlson_late"] - df["charlson_early"]
df["is_left_direction"] = df["difference"] < 0.0
df["is_right_direction"] = df["difference"] > 0.0
df.head()

In [ ]:
bars = alt.Chart().mark_bar(size=10).encode(
    x=alt.X("charlson_early:Q"),
    x2=alt.X2("charlson_late:Q"),
    y=alt.Y("setting:N"),
    color=alt.Color("setting:N")
)

triangle_opacity_scale = scale=alt.Scale(domain=[True, False], range=[1, 0])

triangles_left = alt.Chart().mark_point(shape="triangle-left", filled=True, size=100, xOffset=-4, opacity=1).encode(
    x=alt.X("charlson_late:Q"),
    y=alt.Y("setting:N"),
    color=alt.Color("setting:N"),
    opacity=alt.Opacity("is_left_direction:Q", scale=triangle_opacity_scale, legend=None)
)

triangles_right = alt.Chart().mark_point(shape="triangle-right", filled=True, size=100, xOffset=8, opacity=1).encode(
    x=alt.X("charlson_early:Q"),
    y=alt.Y("setting:N"),
    color=alt.Color("setting:N"),
    opacity=alt.Opacity("is_right_direction:Q", scale=triangle_opacity_scale, legend=None)
)

alt.layer(bars, triangles_left, triangles_right, data=df).facet(
    row=alt.Row("country:N"),
)